# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter as xw
import math
import json
import datetime
from pandas import DataFrame
from datetime import datetime as dt
from alpha_vantage.timeseries import TimeSeries

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
#print(stocks)

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secret import ALPHA_VANTAGE_API_TOKEN
from secret import POLYGON_API_KEY

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
# symbol = 'AAPL'
# company_info_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}'
# price_info_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}'
# print(price_info_url)
# print(company_info_url)
# company_data = requests.get(company_info_url).json()
# price_data = requests.get(price_info_url).json()
# print(price_data)

In [5]:
# price_data = requests.get(price_info_url).json()
# print(price_data)

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
# market_cap = company_data['MarketCapitalization']
# price = price_data['Global Quote']['05. price']
# print(market_cap)
# print(price)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
data_columns = ['Ticker', 'Price', 'Market Cap', 'Shares to Buy']
# final_dataframe = pd.DataFrame(columns = data_columns)
# apple = pd.Series([symbol, price, market_cap, 'N/A'], index = data_columns)
# final_dataframe.loc[len(final_dataframe)] = apple
# final_dataframe

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_df = pd.DataFrame(columns=data_columns)
generator = np.random.default_rng()
for stock in stocks['Ticker']:
    price = generator.integers(50, 500)
    market_cap = generator.integers(50000000000, 2000000000000)
    stock_data = pd.Series([stock, price, market_cap, 'N/A'], index = data_columns)
    final_df.loc[len(final_df)] = stock_data
final_df

,Ticker,Price,Market Cap,Shares to Buy
0,A,246,666989022982,N/A
1,AAL,285,705802435689,N/A
2,AAP,403,1061285709556,N/A
3,AAPL,315,1042147315166,N/A
4,ABBV,292,389518060098,N/A
...,...,...,...,...
500,YUM,220,1769374677079,N/A
501,ZBH,411,498098626870,N/A
502,ZBRA,461,1881150737885,N/A
503,ZION,395,657635225298,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

I am broke and can't afford batch calls cus premium!

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [9]:
portfolio_size = 'none'
while not type(portfolio_size) is float:
    try:
        portfolio_size = float(input('What is your porfolio size?'))
    except ValueError:
        print('Enter a number for your portfolio size.')
amount_per_stock = portfolio_size/len(stocks)
for i in range(len(final_df)):
    final_df.at[i, 'Shares to Buy'] = int(amount_per_stock/final_df.at[i, 'Price'])
final_df

What is your porfolio size? 100000000000


,Ticker,Price,Market Cap,Shares to Buy
0,A,246,666989022982,804958
1,AAL,285,705802435689,694806
2,AAP,403,1061285709556,491364
3,AAPL,315,1042147315166,628634
4,ABBV,292,389518060098,678150
...,...,...,...,...
500,YUM,220,1769374677079,900090
501,ZBH,411,498098626870,481800
502,ZBRA,461,1881150737885,429544
503,ZION,395,657635225298,501315


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [10]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
print(final_df)
final_df.to_excel(writer, sheet_name = 'Recommended Trades', index = False)

    Ticker Price     Market Cap Shares to Buy
0        A   246   666989022982        804958
1      AAL   285   705802435689        694806
2      AAP   403  1061285709556        491364
3     AAPL   315  1042147315166        628634
4     ABBV   292   389518060098        678150
..     ...   ...            ...           ...
500    YUM   220  1769374677079        900090
501    ZBH   411   498098626870        481800
502   ZBRA   461  1881150737885        429544
503   ZION   395   657635225298        501315
504    ZTS   307   637469573858        645015

[505 rows x 4 columns]


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [11]:
background_color = '#0a0a23'
font_color = 'ffffff'

string_format = writer.book.add_format(
    {
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    }
)

integer_format = writer.book.add_format(
    {
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

##### print(type(writer))
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:D', 18, integer_format)


# writer.close()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [12]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Cap', integer_format],
    'D': ['Shares to Buy', integer_format]
}

for k, v in column_formats.items():
    writer.sheets['Recommended Trades'].set_column(f'{k}:{k}', 18, v[1])
    writer.sheets['Recommended Trades'].write(f'{k}1', v[0], v[1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.close();